In [36]:
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import pandas as pd
import requests
from bs4 import BeautifulSoup
from numpy import floor
import statistics
import warnings

format = 2

format_name = ""
if format == 1:
    format_name = "Tests"
elif format == 2:
    format_name = "ODIs"
else:
    format_name = "T20s"

new_zealand_squad = ["Finn Allen","Tom Blundell","Michael Bracewell","Mark Chapman","Devon Conway","Lockie Ferguson","Matt Henry","Kyle Jamieson","Tom Latham","Adam Milne","Daryl Mitchell","Henry Nicholls","Glenn Phillips","Mitchell Santner","Ish Sodhi", "Tim Southee", "Blair Tickner", "Kane Williamson", "Will Young", "Henry Shipley", "Rachin Ravindra", "Chad Bowes"]

bangladesh_squad = ["Afif Hossain", "Ebadot Hossain", "Litton Das", "Mahedi Hasan", "Mehidy Hasan Miraz", "Mohammad Naim", "Mominul Haque", "Mushfiqur Rahim", "Mustafizur Rahman", "Najmul Hossain Shanto", "Nasum Ahmed", "Nurul Hasan", "Shakib Al Hasan", "Shoriful Islam", "Taijul Islam", "Taskin Ahmed", "Yasir Ali", "Towhid Hridoy", "Hasan Mahmud"]

squads = [
    (new_zealand_squad, "New_Zealand"),
    (bangladesh_squad, "Bangladesh"),
]

In [33]:
warnings.filterwarnings("ignore")

def clean_player_name(player_name):
    # Remove any special characters and spaces from the player's name
    return player_name.strip().replace(" ", "+")

def create_batting_csv(player_names, country):
    t = pd.DataFrame()

    for player in player_names:
        print(player + " (bat) ")
        player2=player
        if player2 == 'Rohit Sharma':
            player2 = 'RG Sharma'

        cleaned_player_name = clean_player_name(player2)

        url = f"http://search.espncricinfo.com/ci/content/player/search.html?search={cleaned_player_name}&x=0&y=0"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")

        # Check if any elements are found with the specified class
        player_smry_elements = soup.find_all(class_='ColumnistSmry')
        if not player_smry_elements:
            print(f"Player '{player}' not found. Skipping...")
            continue

        if player2=="Mark Chapman":
            player_id="438563"

        elif player2=="Matt Henry":
            player_id="506612"

        elif player2=="Daryl Mitchell":
            player_id="381743"

        elif player2=="Mahedi Hasan":
            player_id="833087"

        else:
          player_id = str(player_smry_elements[0]).split('.html')[0].split('/')[-1]
        # print(player_id)
        tr = pd.read_html(f'https://stats.espncricinfo.com/ci/engine/player/{player_id}.html?class={format};template=results;type=batting;view=innings')

        # Check if any tables are found on the webpage
        if not tr:
            print(f"Player '{player}' batting statistics not found. Skipping...")
            continue

        tr = tr[3]  # Assuming the required table is at index 3 (verify this according to the webpage)
        tr['Runs'] = tr['Runs'].astype(str)
        tr['Runs'] = tr['Runs'].str.replace(r'\*$', '', regex=True) # Remove asterisk at the end of values
        tr = tr[tr['Runs'] != 'DNB']

        tr['Runs'] = pd.to_numeric(tr['Runs'], errors='coerce')
        tr['BF'] = pd.to_numeric(tr['BF'], errors='coerce')  # Convert values to numeric, replacing non-numeric with NaN
        tr = tr.dropna(subset=['Runs'])
        tr = tr.dropna(subset=['BF'])
        tr = tr.drop(columns=['Unnamed: 9'])
        tr = tr.rename(columns={'Unnamed: 13': 'MatchNo'})
        tr['Dismissal'] = tr['Dismissal'].apply(lambda x: 0 if x == 'not out' else 1)
        tr = tr.rename(columns={'Dismissal': 'Dismissed'})
        tr = tr.dropna()
        tr['Batter'] = player
        t = pd.concat([t, tr])

    # Reset index and drop the previous index column
    t = t.reset_index(drop=True)

    # Save the DataFrame to a CSV file
    csv_filename = f"{country}_bat.csv"
    t.to_csv("/content/Batting_Data/"+csv_filename, index=False)
    t


In [34]:
def clean_player_name(player_name):
    # Remove any special characters and spaces from the player's name
    return player_name.strip().replace(" ", "+")

def create_bowling_csv(player_names, country):
    o = pd.DataFrame()

    for player in player_names:
        print(player + " (bowl) ")
        player2 = player
        if player == 'Rashid Khan':
            player2= 'rashid-khan-793463'
        elif player == 'Rohit Sharma':
            player2 = 'rg sharma'
        elif player2=="Max O'Dowd":
            player2='Max-O-Dowd'

        cleaned_player_name = clean_player_name(player2)

        url = f"http://search.espncricinfo.com/ci/content/player/search.html?search={cleaned_player_name}&x=0&y=0"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")

        # Check if any elements are found with the specified class
        player_smry_elements = soup.find_all(class_='ColumnistSmry')
        if not player_smry_elements:
            print(f"Player '{player}' not found. Skipping...")
            continue

        if player2=="Mark Chapman":
            player_id="438563"

        elif player2=="Matt Henry":
            player_id="506612"

        elif player2=="Daryl Mitchell":
            player_id="381743"

        elif player2=="Mahedi Hasan":
            player_id="833087"

        else:
              player_id = str(player_smry_elements[0]).split('.html')[0].split('/')[-1]

        lr = pd.read_html(f'https://stats.espncricinfo.com/ci/engine/player/{player_id}.html?class={format};template=results;type=bowling;view=innings')

        # Check if any tables are found on the webpage
        if not lr:
            print(f"Player '{player}' bowling statistics not found. Skipping...")
            continue

        lr = lr[3]  # Assuming the required table is at index 3 (verify this according to the webpage)
        lr = lr.drop(columns=['Unnamed: 7'])
        lr = lr.rename(columns={'Unnamed: 11': 'MatchNo'})
        lr['Runs'] = pd.to_numeric(lr['Runs'], errors='coerce')
        lr['Wkts'] = pd.to_numeric(lr['Wkts'], errors='coerce')
        lr['Econ'] = pd.to_numeric(lr['Econ'], errors='coerce')  # Convert values to numeric, replacing non-numeric with NaN
        lr['Overs'] = pd.to_numeric(lr['Overs'], errors='coerce')
        lr = lr.dropna()
        lr['Balls'] = floor(lr['Overs']) * 6 + floor((lr['Overs'] % 1) * 10)
        lr['player'] = player
        o = pd.concat([o, lr])

    # Reset index and drop the previous index column
    o = o.reset_index(drop=True)

    # Save the DataFrame to a CSV file
    csv_filename = f"{country}_bowl.csv"
    o.to_csv("/content/Bowling_Data/"+csv_filename, index=False)

In [37]:
for squad, country in squads:
    print(f"Creating CSV for {country} squad:")
    create_batting_csv(squad, country)
    create_bowling_csv(squad, country)
    print(f"CSV files for {country} squad created.\n")

Creating CSV for New_Zealand squad:
Finn Allen (bat) 
Tom Blundell (bat) 
Michael Bracewell (bat) 
Mark Chapman (bat) 
Devon Conway (bat) 
Lockie Ferguson (bat) 
Matt Henry (bat) 
Kyle Jamieson (bat) 
Tom Latham (bat) 
Adam Milne (bat) 
Daryl Mitchell (bat) 
Henry Nicholls (bat) 
Glenn Phillips (bat) 
Mitchell Santner (bat) 
Ish Sodhi (bat) 
Tim Southee (bat) 
Blair Tickner (bat) 
Kane Williamson (bat) 
Will Young (bat) 
Henry Shipley (bat) 
Rachin Ravindra (bat) 
Chad Bowes (bat) 
Finn Allen (bowl) 
Tom Blundell (bowl) 
Michael Bracewell (bowl) 
Mark Chapman (bowl) 
Devon Conway (bowl) 
Lockie Ferguson (bowl) 
Matt Henry (bowl) 
Kyle Jamieson (bowl) 
Tom Latham (bowl) 
Adam Milne (bowl) 
Daryl Mitchell (bowl) 
Henry Nicholls (bowl) 
Glenn Phillips (bowl) 
Mitchell Santner (bowl) 
Ish Sodhi (bowl) 
Tim Southee (bowl) 
Blair Tickner (bowl) 
Kane Williamson (bowl) 
Will Young (bowl) 
Henry Shipley (bowl) 
Rachin Ravindra (bowl) 
Chad Bowes (bowl) 
CSV files for New_Zealand squad created.